In [ ]:
import firedrake
mesh = firedrake.UnitSquareMesh(32, 32)
degree = 1
temperature_space = firedrake.FunctionSpace(mesh, 'CG', degree)

In [ ]:
from firedrake import exp, Constant
def Θ(x):
    return exp(x) / (exp(x) + exp(-x))

T_0 = Constant(0.0)
δT = Constant(1.0)
x = firedrake.SpatialCoordinate(mesh)
x_0 = Constant(0.25)
α = Constant(8.0)
T_Γ = Θ(α * (x_0 - x[0]))

In [ ]:
pressure_space = firedrake.FunctionSpace(mesh, 'CG', 1)
velocity_space = firedrake.VectorFunctionSpace(mesh, 'CG', 2)
Z = velocity_space * pressure_space
z = firedrake.Function(Z)
u, p = firedrake.split(z)

In [ ]:
from firedrake import inner, grad, dx, ds
ρ = Constant(1)
c = Constant(1)
k = Constant(1e-3)
h = Constant(10 * k)

T = firedrake.Function(temperature_space)
T_n = T.copy(deepcopy=True)
J_mass = 0.5 * ρ * c * (T - T_n)**2 * dx
J_cells = 0.5 * k * inner(grad(T), grad(T)) * dx
J_boundary = 0.5 * h * (T - T_Γ)**2 * ds((3,)) + 0.5 * h * T**2 * ds((1, 2, 4))

δt = Constant(1e-1)
J = J_mass + δt * (J_cells + J_boundary)
F_diffusive = firedrake.derivative(J, T)

ϕ = firedrake.TestFunction(temperature_space)
F_advective = -δt * ρ * c * T * inner(u, grad(ϕ)) * dx

from firedrake import (
    NonlinearVariationalProblem as Problem,
    NonlinearVariationalSolver as Solver,
)

F = F_diffusive + F_advective
temperature_problem = Problem(F, T)
temperature_solver = Solver(temperature_problem)

In [ ]:
from firedrake import MixedVectorSpaceBasis, VectorSpaceBasis
basis = VectorSpaceBasis(constant=True)
nullspace = MixedVectorSpaceBasis(Z, [Z.sub(0), basis])

In [ ]:
from firedrake import sym, div, as_vector
μ = Constant(1e2)
ε = sym(grad(u))
τ = 2 * μ * ε

g = as_vector((0, -1))
f = -T * g
J = (0.5 * inner(τ, ε) - p * div(u) - inner(f, u)) * dx

F = firedrake.derivative(J, z)
bc = firedrake.DirichletBC(Z.sub(0), as_vector((0, 0)), 'on_boundary')
stokes_problem = Problem(F, z, bc)
stokes_solver = Solver(stokes_problem, nullspace=nullspace)

In [ ]:
import tqdm
final_time = 1e2
num_steps = int(final_time / float(δt))
Ts = [T.copy(deepcopy=True)]
zs = [z.copy(deepcopy=True)]
output_freq = 10
for step in tqdm.trange(num_steps):
    temperature_solver.solve()
    stokes_solver.solve()
    T_n.assign(T)
    
    if (step + 1) % output_freq == 0:
        Ts.append(T.copy(deepcopy=True))
        zs.append(z.copy(deepcopy=True))

In [ ]:
%%capture
import matplotlib.pyplot as plt
fig, axes = plt.subplots()
axes.set_aspect('equal')
colors = firedrake.tripcolor(
    Ts[0], num_sample_points=4, vmin=0.0, vmax=0.5, axes=axes
)
fig.colorbar(colors);

In [ ]:
from matplotlib.animation import FuncAnimation
fn_plotter = firedrake.FunctionPlotter(mesh, num_sample_points=4)
def animate(T):
    colors.set_array(fn_plotter(T))
    
interval = 1e3 * output_freq * float(δt) / 10
animation = FuncAnimation(fig, animate, frames=Ts, interval=interval)

In [ ]:
from IPython.display import HTML
HTML(animation.to_jshtml())

In [ ]:
z = zs[-1]
u, p = z.split()
fig, axes = plt.subplots()
axes.set_aspect('equal')
streamlines = firedrake.streamplot(u, axes=axes, seed=1729)
fig.colorbar(streamlines);